In [1]:
#Install + Imports
!pip -q install tensorflow scikit-learn

import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, f1_score, classification_report

import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras import layers, models


In [2]:
#CSV Upload Code
from google.colab import files
files.upload()

Saving tourism_multilingual_1500_FINAL.csv to tourism_multilingual_1500_FINAL.csv


{'tourism_multilingual_1500_FINAL.csv': b'\xef\xbb\xbfid,category,language,question,answer\n1,CATEGORY 1 \xe2\x80\x94 ACCOMMODATION,English,What types of rooms are available at this accommodation?,"We offer single, double, family, and deluxe rooms depending on your preference."\n2,CATEGORY 1 \xe2\x80\x94 ACCOMMODATION,English,What documents do I need to check in?,A passport or national ID card is required for registration.\n3,CATEGORY 1 \xe2\x80\x94 ACCOMMODATION,English,Does the room have hot water?,"Yes, hot water is available 24 hours a day."\n4,CATEGORY 1 \xe2\x80\x94 ACCOMMODATION,English,Are electrical outlets suitable for charging mobile phones?,"Yes, you can charge your mobile devices using the standard outlets provided in the room."\n5,CATEGORY 1 \xe2\x80\x94 ACCOMMODATION,English,Is food available at the accommodation?,"Yes, we provide breakfast, lunch, dinner, and a selection of local meals."\n6,CATEGORY 1 \xe2\x80\x94 ACCOMMODATION,English,Is daily room cleaning included?,"

In [3]:
#Load dataset
CSV_PATH = "tourism_multilingual_1500_FINAL.csv"   # make sure uploaded
df = pd.read_csv(CSV_PATH, encoding="utf-8")
df = df.dropna(subset=["category","language","question","answer"]).copy()

#Use question as input text
df["text"] = df["question"].astype(str).str.strip()
df["category"] = df["category"].astype(str).str.strip()

print("Shape:", df.shape)
print(df["language"].value_counts())
print("Num categories:", df["category"].nunique())
print(df["category"].value_counts())


Shape: (1500, 6)
language
English    500
Sinhala    500
Tamil      500
Name: count, dtype: int64
Num categories: 10
category
CATEGORY 1 — ACCOMMODATION                       150
CATEGORY 2 — TRANSPORTATION                      150
CATEGORY 3 — TOURIST ATTRACTIONS                 150
CATEGORY 4 — FOOD & CULTURE                      150
CATEGORY 5 — HOTEL SERVICES                      150
CATEGORY 6 — SAFETY & EMERGENCY                  150
CATEGORY 7 — SHOPPING & MONEY                    150
CATEGORY 8 — CULTURE, TRADITIONS & EVENTS        150
CATEGORY 9 — WEATHER & SEASONS                   150
CATEGORY 10 — GENERAL TOURIST SAFETY & ADVICE    150
Name: count, dtype: int64


In [4]:
print(df.columns)

Index(['id', 'category', 'language', 'question', 'answer', 'text'], dtype='object')


In [5]:
#If labels are still text
from sklearn.preprocessing import LabelEncoder

label_enc = LabelEncoder()
df["label"] = label_enc.fit_transform(df["category"])


In [6]:
#Train / Val / Test Split

X = df["question"]
y = df["label"]

#80% Train, 20% Temp
X_train, X_temp, y_train, y_temp = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

#Split Temp → 10% Val, 10% Test
X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp,
    test_size=0.5,
    random_state=42,
    stratify=y_temp
)

print("Train size:", len(X_train))
print("Val size  :", len(X_val))
print("Test size :", len(X_test))


Train size: 1200
Val size  : 150
Test size : 150


In [7]:
#Tokenize text + pad sequences
max_words = 30000   #vocab size
max_len   = 40      #sequence length

tokenizer = Tokenizer(num_words=max_words, oov_token="<OOV>")
tokenizer.fit_on_texts(X_train)

X_train = tokenizer.texts_to_sequences(X_train)
X_val   = tokenizer.texts_to_sequences(X_val)
X_test  = tokenizer.texts_to_sequences(X_test)

X_train = pad_sequences(X_train, maxlen=max_len, padding="post", truncating="post")
X_val   = pad_sequences(X_val, maxlen=max_len, padding="post", truncating="post")
X_test  = pad_sequences(X_test, maxlen=max_len, padding="post", truncating="post")


In [8]:
#Build BiLSTM model
embedding_dim = 128
lstm_units = 128

num_classes = len(label_enc.classes_)

model = tf.keras.Sequential([
    layers.Embedding(input_dim=max_words, output_dim=embedding_dim, input_length=max_len),
    layers.Bidirectional(layers.LSTM(lstm_units)),
    layers.Dropout(0.3),
    layers.Dense(128, activation="relu"),
    layers.Dropout(0.3),
    layers.Dense(num_classes, activation="softmax")
])

model.compile(
    loss="sparse_categorical_crossentropy",
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
    metrics=["accuracy"]
)

model.summary()


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional (Bidirectional)   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [9]:
#Train BiLSTM
callbacks = [
    tf.keras.callbacks.EarlyStopping(
        monitor="val_loss",
        patience=2,
        restore_best_weights=True
    )
]

history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),   # IMPORTANT
    epochs=25,
    batch_size=32,
    callbacks=callbacks,
    verbose=1
)


Epoch 1/25
38/38 ━━━━━━━━━━━━━━━━━━━━ 18s 349ms/step - accuracy: 0.0853 - loss: 2.3034 - val_accuracy: 0.1400 - val_loss: 2.2804
Epoch 2/25
38/38 ━━━━━━━━━━━━━━━━━━━━ 12s 131ms/step - accuracy: 0.2050 - loss: 2.2356 - val_accuracy: 0.3267 - val_loss: 1.9291
Epoch 3/25
38/38 ━━━━━━━━━━━━━━━━━━━━ 6s 151ms/step - accuracy: 0.3931 - loss: 1.7176 - val_accuracy: 0.4333 - val_loss: 1.5280
Epoch 4/25
38/38 ━━━━━━━━━━━━━━━━━━━━ 9s 131ms/step - accuracy: 0.5610 - loss: 1.1770 - val_accuracy: 0.5533 - val_loss: 1.2607
Epoch 5/25
38/38 ━━━━━━━━━━━━━━━━━━━━ 6s 160ms/step - accuracy: 0.7712 - loss: 0.6525 - val_accuracy: 0.6067 - val_loss: 1.1969
Epoch 6/25
38/38 ━━━━━━━━━━━━━━━━━━━━ 5s 132ms/step - accuracy: 0.8694 - loss: 0.3619 - val_accuracy: 0.6067 - val_loss: 1.4126
Epoch 7/25
38/38 ━━━━━━━━━━━━━━━━━━━━ 6s 160ms/step - accuracy: 0.9317 - loss: 0.2046 - val_accuracy: 0.5800 - val_loss: 1.5880


In [10]:
#Validation Evaluation
val_pred_prob = model.predict(X_val)
val_pred = np.argmax(val_pred_prob, axis=1)

print("🔹 Validation Accuracy:", accuracy_score(y_val, val_pred))
print("🔹 Validation Macro F1:", f1_score(y_val, val_pred, average="macro"))

5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 108ms/step
🔹 Validation Accuracy: 0.6066666666666667
🔹 Validation Macro F1: 0.6089534796286484


In [11]:
#Test Evaluation (FINAL – BiLSTM)
test_pred_prob = model.predict(X_test)
test_pred = np.argmax(test_pred_prob, axis=1)

print("\n✅ BiLSTM Test Accuracy:", accuracy_score(y_test, test_pred))
print("✅ BiLSTM Test Macro F1:", f1_score(y_test, test_pred, average="macro"))

target_names = [str(x) for x in label_enc.classes_]
print("\nClassification Report (Test Set):\n")
print(classification_report(y_test, test_pred, target_names=target_names))


5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step

✅ BiLSTM Test Accuracy: 0.6066666666666667
✅ BiLSTM Test Macro F1: 0.6110347890744076

Classification Report (Test Set):

                                               precision    recall  f1-score   support

                   CATEGORY 1 — ACCOMMODATION       0.67      0.67      0.67        15
CATEGORY 10 — GENERAL TOURIST SAFETY & ADVICE       0.43      0.20      0.27        15
                  CATEGORY 2 — TRANSPORTATION       0.56      0.67      0.61        15
             CATEGORY 3 — TOURIST ATTRACTIONS       0.67      0.53      0.59        15
                  CATEGORY 4 — FOOD & CULTURE       0.79      0.73      0.76        15
                  CATEGORY 5 — HOTEL SERVICES       0.80      0.80      0.80        15
              CATEGORY 6 — SAFETY & EMERGENCY       0.53      0.53      0.53        15
                CATEGORY 7 — SHOPPING & MONEY       0.31      0.67      0.43        15
    CATEGORY 8 — CULTURE, TRADITIONS & EVENTS       0.9